<h1> Histology + Z scores</h1>

<h2> Read and Format z scores </h2> 
<br>
Data stored wrong way round -> need to transform
names of individuals need to be formated for consistency (name fix)

In [ ]:
### Read in z_scores and format ###
z_scores = read.csv("data_mRNA_median_Zscores.txt", sep = "\t", stringsAsFactors = FALSE, header = TRUE)

map = z_scores[,c(1,2)]

inst_names = colnames(z_scores)[3:length(colnames(z_scores))]

z_scores = z_scores[,-2]
z_scores = as.data.frame(t(z_scores), stringsAsFactors = FALSE) #transformation
names(z_scores) = map[,1]
z_scores = z_scores[-1,]

#convert z scores to numeric datatype
z_scores = as.data.frame(apply(z_scores, 2, as.numeric))

#Fix inconsistent names -> gets rid of "TARGET" and ".01" from instances
name_fix = c()
for(name in inst_names){name_fix = c(name_fix,substr(name,11,nchar(name)-3))}


row.names(z_scores) = name_fix

<h2> Deal with missing values </h2> <br>
The following code deletes genes with > thresh number of NA values. 

In [ ]:
### Parameters ###

thresh = 0

In [ ]:
### Deal with missing vals ###

na_count = colSums(is.na.data.frame(z_scores)) # find NAs in each gene

to_del = c()
for(name in names(z_scores)){
  if(na_count[name]>thresh){
    to_del = c(to_del,name) #make list of genes to be removed
  }
}

z_scores = z_scores[,!(names(z_scores) %in% to_del)] #delete from df using list

<h2> Omit Attributes with majority small z scores </h2> 
<br> Since genes with little up/down regulation will have small absolute z scores, they won't be helpful in class deliniation and can be removed.

In [ ]:
### Parameters ###

instances = nrow(z_scores)
maj = instances %/% 2 #change to alter definition of majority
sig_thresh = 1 #change to alter value z score thresholded at

In [ ]:
### Omit Attributes with majority small z scores ###
smallz = c()
for(name in names(z_scores)){ #for each column
  current_col = z_scores[,name]
  current_count = 0
  for(val in current_col){ #find num abs(z scores) < sig_thresh
    if(abs(val) <= sig_thresh){current_count = current_count + 1}
  }
  if(current_count > maj){smallz = c(smallz,name)} #if count > maj -> remove
}

z_scores = z_scores[,!(names(z_scores) %in% smallz)]

<h2> Load Sample Histology data </h2>  <br> need to format and fix patient naming convention 

In [ ]:
# IMPORTANT: Ignore the first 4 rows of this CSV!
patient = as.data.frame(read.csv("data_clinical_patient.txt", sep = "\t", stringsAsFactors = FALSE, header = TRUE, skip = 4)) 
p_nam = patient[,1]

#Standardise patient names (similar to above)
p_nam_fix = c()
for(name in p_nam){p_nam_fix = c(p_nam_fix,substr(name,11,nchar(name)))}

patient = patient[,2:ncol(patient)]

row.names(patient) = p_nam_fix

#Isolate just the histology column:
TSH = data.frame(patient[,"TUMOR_SAMPLE_HISTOLOGY"])
colnames(TSH) = "Histology"
rownames(TSH) = p_nam_fix

#See how many of the patients shared between two datasets
rownames(z_scores) %in% rownames(TSH)

<h2> Merge Datasets </h2>

In [ ]:
mg = merge.data.frame(TSH,z_scores, by="row.names")

cert = mg[!(mg$Histology == "Unknown"),] # remove all unknown hisotologies

<h2>Write to a CSV </h2>

In [ ]:
write.csv(cert,file = "z_score_hist.csv") # write to a csv

<h2> Next Steps </h2>

Classification performed with WEKA -> next step is to perform this either in R or with scikit learn